We organized the ISLES Challenge in the blue_isc5935/share/team02/data directory (in the HiPerGator environment) into masks, NIFTIs, and processedNIFTIs directories. The NIFTIs directory has 3 NIFTI files for each subject, one NIFTI for each MRI modality. Originally, these files were in folders for each subject, along with some .json files. I moved the .nii files so that they were all in the same directory, making it easy to load those files in all at once. There may be a need to differentiate these files based on modality, in which case the file names can be checked. Similarly, the masks were placed in individual directories for each subject along with some other derivative information, so I isolated all those masks files into one directory, thus making it easy to load in that data if necessary. In anticpation of needing to remain organized after any preprocessing, I made an empty processedNIFTIs directory to store any future preprocessed files. ISLES only provides a training dataset, but we will likely split the existing training dataset into training, validation, and test sets, in which case we will use a split data function to do so. It seems that this can be done when loading in the data with relatively little complexity. Splitting up the data while loading can also allow us to change the training, validation, and test set proportions by using a function.